In [4]:
# %%capture captured

# -*- coding: utf-8 -*-
"""EEG_Random_Forest_Training.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1FVV8kiLLHNa0W8ekMltR-x8UuECxWO0r
"""

import pandas as pd
import numpy as np
import matplotlib as plt

"""## **EEG FEATURES MOUNTING**"""

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the components of your path
drive_base = '/content/drive'
my_drive = 'MyDrive'
project_folder = 'EEG PE'
target_folder = 'Users_data'
# subject = 17
for subject in range(16,31):
  # Construct the full path using os.path.join
  full_path = os.path.join(drive_base, my_drive, project_folder, target_folder,str(subject))
  # print("Constructed path:", full_path)

  # Check if the folder exists
  if os.path.exists(full_path):
      # print(f"Folder exists at: {full_path}")
      os.chdir(full_path)
  #     print("Changed directory to:", os.getcwd())
  # else:
  #     print(f"Folder does not exist at: {full_path}")

  # List of filenames for each dataset
  p_files = ["features_1_1.csv", "features_1_4.csv", "features_1_5.csv", "features_1_6.csv", "features_1_7.csv", "features_1_8.csv"]
  c_files = ["features_2_1.csv", "features_2_4.csv", "features_2_5.csv", "features_2_6.csv", "features_2_7.csv", "features_2_8.csv"]

  # Function to read CSV files into a list of DataFrames
  def read_csv_files(file_list):
      dataframes = []
      for file in file_list:
          df = pd.read_csv(file)
          # Extract just the time part of 'ftime' and store it in a new column
          df['ftime_only'] = pd.to_datetime(df['ftime']).dt.strftime('%H:%M:%S')
          dataframes.append(df)
      return dataframes


  # Read all files into lists of DataFrames
  p_data = read_csv_files(p_files)
  c_data = read_csv_files(c_files)

  # Function to read CSV files into a list of DataFrames
  def read_csv_files(file_list):
      return [pd.read_csv(file) for file in file_list]

  # Read all files into lists of DataFrames
  p_data = read_csv_files(p_files)
  c_data = read_csv_files(c_files)

  # Function to merge DataFrames in pairs
  def merge_dataframes(data_list):
      merged_data = data_list[0]
      for i in range(1, len(data_list)):
          merged_data = pd.merge(merged_data, data_list[i], how="inner", on=["ftime"], suffixes=('', f'_file{i}'))

      return merged_data

  # Merge DataFrames for each dataset
  p_merged = merge_dataframes(p_data)
  c_merged = merge_dataframes(c_data)

  # Function to clean the merged DataFrame
  def clean_dataframe(df):
      # Ensure 'ftime' is retained and formatted
      if 'ftime' in df.columns:
          df['ftime'] = pd.to_datetime(df['ftime']).dt.strftime('%H:%M:%S')

      df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Drop 'Unnamed' columns
      df = df.replace(np.nan, 0)                            # Replace NaNs with 0
      df = df.replace(np.inf, 10000)                        # Replace +Inf with a large value
      df = df.replace(-np.inf, -999999)                     # Replace -Inf with a large negative value

      return df

  # Clean the merged DataFrames
  pmerge5 = clean_dataframe(p_merged)
  cmerge5 = clean_dataframe(c_merged)

  cmerge5['ftime'] = pd.to_datetime(cmerge5['ftime']).dt.strftime('%H:%M:%S')
  pmerge5['ftime'] = pd.to_datetime(pmerge5['ftime']).dt.strftime('%H:%M:%S')

  # print(cmerge5.shape)
  # print(pmerge5.shape)

  """# Writing Timestamps

  """

  timestampsfoldername = 'Timestamps'
  stampsfolder = 'stamps'
  syncedtimecsvfile = 'Synced_Times_All.csv'

  full_path = os.path.join(drive_base, my_drive, project_folder, timestampsfoldername,stampsfolder)
  # print("Constructed path:", full_path)

  # Check if the folder exists
  if os.path.exists(full_path):
      # print(f"Folder exists at: {full_path}")
      os.chdir(full_path)
  #     print("Changed directory to:", os.getcwd())
  # else:
  #     print(f"Folder does not exist at: {full_path}")

  df = pd.read_csv(syncedtimecsvfile)

  offsets = {}
  for index, row in df.iterrows():
      # Assuming the participant number is extracted from the 'File Name' and matches the participant id directly
      participant_id = int(row['File Name'][:2]) # Modify this line if the format is different

      # Extracting and converting the offset time from '(hh, mm, ss)' to 'hh:mm:ss'
      offset_time = row['Offset Time']
      offset_time_clean = offset_time.strip('()')  # Remove parentheses
      hh, mm, ss = map(int, offset_time_clean.split(', '))  # Split and convert to integers
      formatted_time = f"{hh:02d}:{mm:02d}:{ss:02d}"  # Format to 'hh:mm:ss'

      offsets[participant_id] = formatted_time

  #Construct the full path using os.path.join
  full_path = os.path.join(drive_base, my_drive, project_folder, target_folder,str(subject))
  # print("Constructed path:", full_path)

  # Check if the folder exists
  if os.path.exists(full_path):
      # print(f"Folder exists at: {full_path}")
      os.chdir(full_path)
  #     print("Changed directory to:", os.getcwd())
  # else:
  #     print(f"Folder does not exist at: {full_path}")
  offsets

  import pandas as pd
  from datetime import timedelta

  full_path = os.path.join(drive_base, my_drive, project_folder, timestampsfoldername,stampsfolder)
  # print("Constructed path:", full_path)

  # Check if the folder exists
  if os.path.exists(full_path):
      # print(f"Folder exists at: {full_path}")
      os.chdir(full_path)
  #     print("Changed directory to:", os.getcwd())
  # else:
  #     print(f"Folder does not exist at: {full_path}")

  # Load CSV file
  df = pd.read_csv('trainingStamps.csv')

  # Strip leading/trailing spaces from column names
  df.columns = df.columns.str.strip()

  # Replace invalid values like '-' with NaN
  # df['Start TIme (minutes)'] = df['Start'].replace('-', pd.NA)
  # df['End Time (minutes)'] = df['End'].replace('-', pd.NA)

  df['Start TIme (minutes)'] = df['Start']
  df['End Time (minutes)'] = df['End']

  df = df.drop('Start', axis=1)
  df = df.drop('End', axis=1)

  # Convert valid time strings to datetime, using errors='coerce' to handle invalid formats
  # df['Start TIme (minutes)'] = pd.to_datetime(df['Start TIme (minutes)'], format='%M:%S', errors='coerce').dt.time
  # df['End Time (minutes)'] = pd.to_datetime(df['End Time (minutes)'], format='%M:%S', errors='coerce').dt.time

  # Display the first 5 rows and the column names
  # print(df.head(5))
  # print(df.columns)

  #Construct the full path using os.path.join
  full_path = os.path.join(drive_base, my_drive, project_folder, target_folder,str(subject))
  # print("Constructed path:", full_path)

  # Check if the folder exists
  if os.path.exists(full_path):
      # print(f"Folder exists at: {full_path}")
      os.chdir(full_path)
      # print("Changed directory to:", os.getcwd())
  # else:
      # print(f"Folder does not exist at: {full_path}")

  """# TRAINING DATA TIMESTAMPS

  ## Writing timestamps extracted from the excel
  """

  from datetime import timedelta, datetime

  # Filter rows for the specific subject
  training_time_df = df[df['Subject'] == subject]

  # Function to convert offset string to timedelta
  def offset_to_timedelta(offset_str):
      h, m, s = map(int, offset_str.split(':'))
      return timedelta(hours=h, minutes=m, seconds=s)

  def string_to_time(time_str):
      return datetime.strptime(time_str, '%H:%M:%S').time()

  # Apply the offset for the selected subject
  if subject in offsets:
      time_offset = offset_to_timedelta(offsets[subject])

      # Add the offset to the 'Start Time (minutes)' and 'End Time (minutes)' columns
      training_time_df['Start TIme (minutes)'] = training_time_df['Start TIme (minutes)'].apply(
          lambda t: (datetime.combine(datetime.today(), string_to_time(t)) + time_offset).time() if pd.notna(t) else t
      )
      training_time_df['End Time (minutes)'] = training_time_df['End Time (minutes)'].apply(
          lambda t: (datetime.combine(datetime.today(), string_to_time(t)) + time_offset).time() if pd.notna(t) else t
      )

  # print(training_time_df.shape)

  cmerge5['ftime'] = pd.to_datetime(cmerge5['ftime'], format='%H:%M:%S')
  training_time_df['Start TIme (minutes)'] = pd.to_datetime(training_time_df['Start TIme (minutes)'], format='%H:%M:%S')
  training_time_df['End Time (minutes)'] = pd.to_datetime(training_time_df['End Time (minutes)'], format='%H:%M:%S')

  filtered_cmerge5 = pd.DataFrame()

  # Loop over each row in training_time_df
  for _, row in training_time_df.iterrows():
      if row['File'] != '3.py' and row['Task'] == 2 :
        start_time = row['Start TIme (minutes)']
        end_time = row['End Time (minutes)']

        # Filter cmerge5 for rows where 'ftime' is within the time interval
        filtered_rows = cmerge5[(cmerge5['ftime'] >= start_time) & (cmerge5['ftime'] <= end_time)]

        # Append the filtered rows to the result dataframe
        filtered_cmerge5 = pd.concat([filtered_cmerge5, filtered_rows])

  # Reset index after filtering
  filtered_cmerge5 = filtered_cmerge5.reset_index(drop=True)
  filtered_cmerge5 = filtered_cmerge5.drop(columns='ftime')
  filtered_cmerge5['label'] = 1
  # print(filtered_cmerge5.shape)

  pmerge5['ftime'] = pd.to_datetime(pmerge5['ftime'], format='%H:%M:%S')
  training_time_df['Start TIme (minutes)'] = pd.to_datetime(training_time_df['Start TIme (minutes)'], format='%H:%M:%S')
  training_time_df['End Time (minutes)'] = pd.to_datetime(training_time_df['End Time (minutes)'], format='%H:%M:%S')

  filtered_cmerge5_flag0 = pd.DataFrame()

  # Loop over each row in training_time_df
  for _, row in training_time_df.iterrows():
      if row['Task'] == 1:
        start_time = row['Start TIme (minutes)']
        end_time = row['End Time (minutes)']


        # Filter pmerge5 for rows where 'ftime' is within the time interval
        filtered_rows = pmerge5[(pmerge5['ftime'] >= start_time) & (pmerge5['ftime'] <= end_time)]

        # Append the filtered rows to the result dataframe
        filtered_cmerge5_flag0 = pd.concat([filtered_cmerge5_flag0, filtered_rows])

  # Reset index after filtering
  filtered_cmerge5_flag0 = filtered_cmerge5_flag0.reset_index(drop=True)
  # print(filtered_cmerge5_flag0)
  filtered_cmerge5_flag0 = filtered_cmerge5_flag0.drop(columns='ftime')
  filtered_cmerge5_flag0['label'] = 0
  # print(filtered_cmerge5_flag0.shape)

  """## Model Training Dataset"""

  combined_df = pd.concat([filtered_cmerge5, filtered_cmerge5_flag0], ignore_index=True)
  shuffled_df = combined_df.sample(frac=1).reset_index(drop=True)
  dataset = shuffled_df.copy()
  # print(dataset.shape)

  X = dataset.iloc[:, :-1].values
  y = dataset.iloc[:,-1].values

  """## Splitting the dataset into the Training set and Test set"""

  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

  """## Feature Scaling"""

  from sklearn.preprocessing import StandardScaler
  sc = StandardScaler()
  X_train = sc.fit_transform(X_train)
  X_test = sc.transform(X_test)

  """## Training the Random Forest Classification model on the Training set"""

  from sklearn.ensemble import RandomForestClassifier
  classifier = RandomForestClassifier(n_estimators=100, random_state=0)
  classifier.fit(X_train, y_train)

  """## Predicting the Test set results"""

  y_pred = classifier.predict(X_test)
  # print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

  """## Making the Confusion Matrix"""

  from sklearn.metrics import confusion_matrix, accuracy_score
  cm = confusion_matrix(y_test, y_pred)
  # print(cm)
  accuracy_score(y_test, y_pred)

  """# Test Set"""

  import pandas as pd

  test1 = pd.DataFrame()

  # Loop over each row in training_time_df
  for _, row in training_time_df.iterrows():

      if (row['File'] != '1.py' and row['Task'] == 1):
        start_time = row['Start TIme (minutes)']
        end_time = row['End Time (minutes)']

        # Filter cmerge5 for rows where 'ftime' is within the time interval
        filtered_rows = pmerge5[(pmerge5['ftime'] >= start_time) & (pmerge5['ftime'] <= end_time)]

        # Append the filtered rows to the result dataframe
        test1 = pd.concat([test1, filtered_rows])

      if (row['File'] == '1.py' and row['Task'] == 2):
          start_time = row['Start TIme (minutes)']
          end_time = row['End Time (minutes)']

          # Filter cmerge5 for rows where 'ftime' is within the time interval
          filtered_rows = cmerge5[(cmerge5['ftime'] >= start_time) & (cmerge5['ftime'] <= end_time)]

          # Append the filtered rows to the result dataframe
          test1 = pd.concat([test1, filtered_rows])


  # Reset index after filtering
  test1 = test1.reset_index(drop=True)

  # Optionally, drop 'ftime' column and add 'label' column if needed
  test1 = test1.drop(columns='ftime')
  # print(test1.shape)

  test1 = sc.transform(test1)
  y_pred_unknown = classifier.predict(test1)
  # print(y_pred_unknown)

  accuracy = accuracy_score(y_pred_unknown, np.zeros(len(y_pred_unknown)))

  # %%capture captured
  print(f"Accuracy: {accuracy * 100:.2f}%, subject={subject}")
  # captured.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-06043b25fc39>:99: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cmerge5['ftime'] = pd.to_datetime(cmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pmerge5['ftime'] = pd.to_datetime(pmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_time_df['Start TIme (minutes)'] = training_time_df['Start TIme (minute

Accuracy: 66.67%, subject=16


<ipython-input-4-06043b25fc39>:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pmerge5['ftime'] = pd.to_datetime(pmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_time_df['Start TIme (minutes)'] = training_time_df['Start TIme (minutes)'].apply(
<ipython-input-4-06043b25fc39>:228: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

Accuracy: 62.50%, subject=17


...

<ipython-input-4-06043b25fc39>:99: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cmerge5['ftime'] = pd.to_datetime(cmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pmerge5['ftime'] = pd.to_datetime(pmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_time_df['Start TIme (minutes)'] = training_time_df['Start TIme (minute

Accuracy: 53.33%, subject=28


<ipython-input-4-06043b25fc39>:99: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cmerge5['ftime'] = pd.to_datetime(cmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:100: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pmerge5['ftime'] = pd.to_datetime(pmerge5['ftime']).dt.strftime('%H:%M:%S')
<ipython-input-4-06043b25fc39>:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_time_df['Start TIme (minutes)'] = training_time_df['Start TIme (minute

Accuracy: 100.00%, subject=29
Accuracy: 44.83%, subject=30


<ipython-input-4-06043b25fc39>:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_time_df['Start TIme (minutes)'] = pd.to_datetime(training_time_df['Start TIme (minutes)'], format='%H:%M:%S')
<ipython-input-4-06043b25fc39>:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_time_df['End Time (minutes)'] = pd.to_datetime(training_time_df['End Time (minutes)'], format='%H:%M:%S')
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler 